# Lesson I1: Structs & Methods

**Duration**: 90-105 minutes  
**Stage**: Intermediate (Building Skills)  
**Prerequisites**: All Beginner lessons (B1-B3)

---

## 📋 What You'll Learn

This lesson teaches you how to create custom data types with structs and add behavior through methods. You'll learn to organize code using Rust's powerful type system and implement object-oriented patterns in a Rust-idiomatic way.

**Why this matters**: Structs and methods are the foundation of Rust's approach to data modeling. They're used in every Rust application, from web servers to embedded systems.

---

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:
1. Define and use structs with various field types
2. Implement methods and associated functions for structs
3. Create and pattern match enums
4. Work with Option<T> and Result<T,E> types
5. Use Vec, String, and HashMap collections
6. Understand ownership with collections


In [ ]:
// Define a struct to represent a user
#[derive(Debug)]  // Allows us to print the struct
struct User {
    username: String,
    email: String,
    age: u32,
    active: bool,
}

fn basic_struct_usage() {
    // Create an instance
    let user1 = User {
        username: String::from("alice123"),
        email: String::from("alice@example.com"),
        age: 25,
        active: true,
    };
    
    println!("User: {:?}", user1);
    println!("Username: {}", user1.username);
    println!("Email: {}", user1.email);
    
    // Mutable struct
    let mut user2 = User {
        username: String::from("bob456"),
        email: String::from("bob@example.com"),
        age: 30,
        active: false,
    };
    
    // Modify a field
    user2.active = true;
    user2.age += 1;
    
    println!("Updated user: {:?}", user2);
}

basic_struct_usage();

In [ ]:
#[derive(Debug)]
struct Rectangle {
    width: f64,
    height: f64,
}

impl Rectangle {
    // Associated function (constructor)
    fn new(width: f64, height: f64) -> Self {
        Rectangle { width, height }  // Field init shorthand
    }
    
    // Associated function for creating a square
    fn square(size: f64) -> Self {
        Rectangle {
            width: size,
            height: size,
        }
    }
    
    // Method that borrows self immutably
    fn area(&self) -> f64 {
        self.width * self.height
    }
    
    // Method that borrows self immutably
    fn perimeter(&self) -> f64 {
        2.0 * (self.width + self.height)
    }
    
    // Method that borrows self immutably
    fn can_hold(&self, other: &Rectangle) -> bool {
        self.width >= other.width && self.height >= other.height
    }
    
    // Method that borrows self mutably
    fn scale(&mut self, factor: f64) {
        self.width *= factor;
        self.height *= factor;
    }
    
    // Method that takes ownership of self
    fn into_square(self) -> Rectangle {
        let size = self.width.max(self.height);
        Rectangle::square(size)
    }
}

fn struct_methods_demo() {
    // Using associated functions
    let rect1 = Rectangle::new(10.0, 20.0);
    let square1 = Rectangle::square(15.0);
    
    println!("Rectangle: {:?}", rect1);
    println!("Square: {:?}", square1);
    
    // Using methods
    println!("Rectangle area: {:.2}", rect1.area());
    println!("Rectangle perimeter: {:.2}", rect1.perimeter());
    println!("Square area: {:.2}", square1.area());
    
    // Method with multiple parameters
    println!("Can rectangle hold square? {}", rect1.can_hold(&square1));
    println!("Can square hold rectangle? {}", square1.can_hold(&rect1));
    
    // Mutable method
    let mut rect2 = Rectangle::new(5.0, 8.0);
    println!("Before scaling: {:?}", rect2);
    rect2.scale(2.0);
    println!("After scaling: {:?}", rect2);
    
    // Method that takes ownership
    let rect3 = Rectangle::new(3.0, 7.0);
    let new_square = rect3.into_square();  // rect3 is moved here
    println!("Converted to square: {:?}", new_square);
    // rect3 is no longer accessible
}

struct_methods_demo();

### Multiple Implementation Blocks

In [ ]:
#[derive(Debug, Clone)]
struct BankAccount {
    account_number: String,
    holder_name: String,
    balance: f64,
    is_active: bool,
}

// First impl block: Constructors and basic operations
impl BankAccount {
    fn new(account_number: String, holder_name: String) -> Self {
        BankAccount {
            account_number,
            holder_name,
            balance: 0.0,
            is_active: true,
        }
    }
    
    fn balance(&self) -> f64 {
        self.balance
    }
    
    fn is_active(&self) -> bool {
        self.is_active
    }
}

// Second impl block: Transaction operations
impl BankAccount {
    fn deposit(&mut self, amount: f64) -> Result<(), String> {
        if !self.is_active {
            return Err("Account is not active".to_string());
        }
        
        if amount <= 0.0 {
            return Err("Deposit amount must be positive".to_string());
        }
        
        self.balance += amount;
        Ok(())
    }
    
    fn withdraw(&mut self, amount: f64) -> Result<(), String> {
        if !self.is_active {
            return Err("Account is not active".to_string());
        }
        
        if amount <= 0.0 {
            return Err("Withdrawal amount must be positive".to_string());
        }
        
        if amount > self.balance {
            return Err("Insufficient funds".to_string());
        }
        
        self.balance -= amount;
        Ok(())
    }
}

// Third impl block: Account management
impl BankAccount {
    fn close_account(&mut self) {
        self.is_active = false;
    }
    
    fn reopen_account(&mut self) {
        self.is_active = true;
    }
    
    fn transfer_to(&mut self, other: &mut BankAccount, amount: f64) -> Result<(), String> {
        self.withdraw(amount)?;  // ? operator propagates errors
        
        match other.deposit(amount) {
            Ok(()) => Ok(()),
            Err(e) => {
                // Rollback the withdrawal
                self.balance += amount;
                Err(format!("Transfer failed: {}", e))
            }
        }
    }
}

fn multiple_impl_blocks_demo() {
    let mut account1 = BankAccount::new(
        "ACC001".to_string(),
        "Alice Johnson".to_string()
    );
    
    let mut account2 = BankAccount::new(
        "ACC002".to_string(),
        "Bob Smith".to_string()
    );
    
    println!("Initial accounts:");
    println!("Account 1: {:?}", account1);
    println!("Account 2: {:?}", account2);
    
    // Deposit money
    match account1.deposit(1000.0) {
        Ok(()) => println!("Deposited $1000 to account 1"),
        Err(e) => println!("Deposit failed: {}", e),
    }
    
    match account2.deposit(500.0) {
        Ok(()) => println!("Deposited $500 to account 2"),
        Err(e) => println!("Deposit failed: {}", e),
    }
    
    println!("\nAfter deposits:");
    println!("Account 1 balance: ${:.2}", account1.balance());
    println!("Account 2 balance: ${:.2}", account2.balance());
    
    // Transfer money
    match account1.transfer_to(&mut account2, 200.0) {
        Ok(()) => println!("\nTransferred $200 from account 1 to account 2"),
        Err(e) => println!("\nTransfer failed: {}", e),
    }
    
    println!("\nAfter transfer:");
    println!("Account 1 balance: ${:.2}", account1.balance());
    println!("Account 2 balance: ${:.2}", account2.balance());
    
    // Try to withdraw too much
    match account1.withdraw(1000.0) {
        Ok(()) => println!("Withdrawal successful"),
        Err(e) => println!("\nWithdrawal failed: {}", e),
    }
}

multiple_impl_blocks_demo();

In [ ]:
#[derive(Debug, Clone)]
struct Config {
    debug: bool,
    max_connections: u32,
    timeout_seconds: u64,
    server_name: String,
    port: u16,
}

impl Config {
    fn default() -> Self {
        Config {
            debug: false,
            max_connections: 100,
            timeout_seconds: 30,
            server_name: String::from("localhost"),
            port: 8080,
        }
    }
    
    // Constructor using field init shorthand
    fn new(server_name: String, port: u16, debug: bool) -> Self {
        Config {
            server_name,  // Shorthand for server_name: server_name
            port,         // Shorthand for port: port
            debug,        // Shorthand for debug: debug
            ..Config::default()  // Use default values for remaining fields
        }
    }
    
    fn production_config(&self) -> Self {
        Config {
            debug: false,
            max_connections: 1000,
            timeout_seconds: 60,
            ..self.clone()  // Copy other fields from self
        }
    }
}

fn struct_update_syntax_demo() {
    // Default configuration
    let default_config = Config::default();
    println!("Default config: {:?}", default_config);
    
    // Custom configuration using field init shorthand
    let dev_config = Config::new(
        String::from("dev.example.com"),
        3000,
        true
    );
    println!("\nDev config: {:?}", dev_config);
    
    // Production configuration using struct update syntax
    let prod_config = dev_config.production_config();
    println!("\nProduction config: {:?}", prod_config);
    
    // Create a variant with just one field changed
    let test_config = Config {
        port: 4000,
        ..default_config  // Use all other fields from default_config
    };
    println!("\nTest config: {:?}", test_config);
    
    // Note: default_config was moved in the previous line
    // This would cause an error: println!("{:?}", default_config);
}

struct_update_syntax_demo();

In [ ]:
// TODO: Complete the Student struct and its implementation

#[derive(Debug, Clone)]
struct Student {
    id: u32,
    name: String,
    email: String,
    grades: Vec<f64>,
}

impl Student {
    // TODO: Implement constructor
    fn new(id: u32, name: String, email: String) -> Self {
        // Create a new student with empty grades
        Student {
            id,
            name,
            email,
            grades: Vec::new(),
        }
    }
    
    // TODO: Add a grade
    fn add_grade(&mut self, grade: f64) -> Result<(), String> {
        // Validate grade is between 0.0 and 100.0
        if grade < 0.0 || grade > 100.0 {
            return Err("Grade must be between 0.0 and 100.0".to_string());
        }
        self.grades.push(grade);
        Ok(())
    }
    
    // TODO: Calculate average grade
    fn average_grade(&self) -> Option<f64> {
        if self.grades.is_empty() {
            None
        } else {
            let sum: f64 = self.grades.iter().sum();
            Some(sum / self.grades.len() as f64)
        }
    }
    
    // TODO: Get letter grade based on average
    fn letter_grade(&self) -> Option<char> {
        self.average_grade().map(|avg| {
            match avg {
                90.0..=100.0 => 'A',
                80.0..=89.9 => 'B',
                70.0..=79.9 => 'C',
                60.0..=69.9 => 'D',
                _ => 'F',
            }
        })
    }
    
    // TODO: Check if student is passing (average >= 60.0)
    fn is_passing(&self) -> bool {
        self.average_grade().map_or(false, |avg| avg >= 60.0)
    }
}

fn student_management_demo() {
    let mut student = Student::new(
        12345,
        "Alice Johnson".to_string(),
        "alice@university.edu".to_string()
    );
    
    println!("New student: {:?}", student);
    
    // Add some grades
    let grades = [85.5, 92.0, 78.5, 88.0, 95.5];
    for grade in &grades {
        match student.add_grade(*grade) {
            Ok(()) => println!("Added grade: {}", grade),
            Err(e) => println!("Failed to add grade: {}", e),
        }
    }
    
    // Display student information
    println!("\nStudent grades: {:?}", student.grades);
    
    if let Some(avg) = student.average_grade() {
        println!("Average grade: {:.2}", avg);
    }
    
    if let Some(letter) = student.letter_grade() {
        println!("Letter grade: {}", letter);
    }
    
    println!("Is passing: {}", student.is_passing());
    
    // Try to add an invalid grade
    match student.add_grade(105.0) {
        Ok(()) => println!("Added invalid grade"),
        Err(e) => println!("\nError adding invalid grade: {}", e),
    }
}

student_management_demo();

In [ ]:
// TODO: Implement an inventory management system

#[derive(Debug, Clone)]
struct Product {
    sku: String,
    name: String,
    category: String,
    price: f64,
    quantity: u32,
}

#[derive(Debug)]
struct Inventory {
    warehouse_name: String,
    products: Vec<Product>,
}

// TODO: Implement methods for Product
impl Product {
    fn new(sku: String, name: String, category: String, price: f64, quantity: u32) -> Self {
        // TODO: Create a new product
        Product {
            sku,
            name,
            category,
            price,
            quantity,
        }
    }
    
    fn restock(&mut self, amount: u32) {
        // TODO: Add stock to the product
        self.quantity += amount;
    }
    
    fn sell(&mut self, amount: u32) -> Result<f64, String> {
        // TODO: Sell product and return total price, or error if insufficient stock
        if self.quantity < amount {
            Err(format!("Insufficient stock. Available: {}, Requested: {}", self.quantity, amount))
        } else {
            self.quantity -= amount;
            Ok(self.price * amount as f64)
        }
    }
    
    fn is_low_stock(&self, threshold: u32) -> bool {
        // TODO: Check if product is below stock threshold
        self.quantity < threshold
    }
    
    fn total_value(&self) -> f64 {
        // TODO: Calculate total value of product in stock
        self.price * self.quantity as f64
    }
}

// TODO: Implement methods for Inventory
impl Inventory {
    fn new(warehouse_name: String) -> Self {
        // TODO: Create a new inventory with empty product list
        Inventory {
            warehouse_name,
            products: Vec::new(),
        }
    }
    
    fn add_product(&mut self, product: Product) {
        // TODO: Add a product to the inventory
        self.products.push(product);
    }
    
    fn find_product_by_sku(&mut self, sku: &str) -> Option<&mut Product> {
        // TODO: Find a product by SKU
        self.products.iter_mut().find(|product| product.sku == sku)
    }
    
    fn products_by_category(&self, category: &str) -> Vec<&Product> {
        // TODO: Return products in a specific category
        self.products.iter().filter(|product| product.category == category).collect()
    }
    
    fn low_stock_products(&self, threshold: u32) -> Vec<&Product> {
        // TODO: Return products below stock threshold
        self.products.iter().filter(|product| product.is_low_stock(threshold)).collect()
    }
    
    fn total_inventory_value(&self) -> f64 {
        // TODO: Calculate total value of all products
        self.products.iter().map(|product| product.total_value()).sum()
    }
}

fn inventory_management_demo() {
    let mut inventory = Inventory::new("Main Warehouse".to_string());
    
    // Add some products
    inventory.add_product(Product::new(
        "LAPTOP-001".to_string(),
        "ThinkPad X1 Carbon".to_string(),
        "Electronics".to_string(),
        1299.99,
        15
    ));
    
    inventory.add_product(Product::new(
        "MOUSE-042".to_string(),
        "Wireless Mouse".to_string(),
        "Electronics".to_string(),
        29.99,
        50
    ));
    
    inventory.add_product(Product::new(
        "DESK-101".to_string(),
        "Standing Desk".to_string(),
        "Furniture".to_string(),
        499.99,
        8
    ));
    
    println!("Warehouse: {}", inventory.warehouse_name);
    println!("Total products: {}", inventory.products.len());
    println!("Total inventory value: ${:.2}", inventory.total_inventory_value());
    
    // Show electronics
    let electronics = inventory.products_by_category("Electronics");
    println!("\nElectronics products: {}", electronics.len());
    for product in &electronics {
        println!("  - {} (Stock: {}, Price: ${:.2})", product.name, product.quantity, product.price);
    }
    
    // Sell a product
    if let Some(product) = inventory.find_product_by_sku("LAPTOP-001") {
        match product.sell(3) {
            Ok(total) => println!("\nSold 3 units of {} for ${:.2}", product.name, total),
            Err(e) => println!("\nFailed to sell: {}", e),
        }
    }
    
    // Check low stock
    let low_stock = inventory.low_stock_products(10);
    println!("\nLow stock products (< 10 units): {}", low_stock.len());
    for product in &low_stock {
        println!("  - {} (Stock: {})", product.name, product.quantity);
    }
}

inventory_management_demo();

In [ ]:
// Simple enum without data
#[derive(Debug, PartialEq)]
enum Direction {
    North,
    South,
    East,
    West,
}

// Enum with data-carrying variants
#[derive(Debug)]
enum Message {
    Quit,                       // No data
    Move { x: i32, y: i32 },    // Struct-like variant
    Write(String),              // Tuple-like variant
    ChangeColor(u8, u8, u8),    // Multiple values
}

fn basic_enum_demo() {
    // Using simple enum
    let direction = Direction::North;
    
    match direction {
        Direction::North => println!("Going north! ⬆️"),
        Direction::South => println!("Going south! ⬇️"),
        Direction::East => println!("Going east! ➡️"),
        Direction::West => println!("Going west! ⬅️"),
    }
    
    // Using data-carrying enum
    let messages = vec![
        Message::Write(String::from("Hello, World!")),
        Message::Move { x: 10, y: 20 },
        Message::ChangeColor(255, 0, 128),
        Message::Quit,
    ];
    
    for message in messages {
        match message {
            Message::Quit => {
                println!("Received quit message");
                break;
            },
            Message::Move { x, y } => {
                println!("Move to coordinates: ({}, {})", x, y);
            },
            Message::Write(text) => {
                println!("Text message: {}", text);
            },
            Message::ChangeColor(r, g, b) => {
                println!("Change color to RGB({}, {}, {})", r, g, b);
            },
        }
    }
}

basic_enum_demo();

### Advanced Pattern Matching Techniques

In [ ]:
#[derive(Debug)]
enum WebEvent {
    PageLoad,
    PageUnload,
    KeyPress(char),
    Paste(String),
    Click { x: i64, y: i64 },
}

#[derive(Debug)]
struct Point {
    x: i32,
    y: i32,
}

fn advanced_pattern_matching() {
    let events = vec![
        WebEvent::Click { x: 100, y: 200 },
        WebEvent::KeyPress('a'),
        WebEvent::KeyPress('1'),
        WebEvent::Paste(String::from("Hello")),
        WebEvent::Click { x: 0, y: 0 },
    ];
    
    for event in events {
        match event {
            // Match guards - additional conditions
            WebEvent::Click { x, y } if x == 0 && y == 0 => {
                println!("Clicked at origin!");
            },
            WebEvent::Click { x, y } if x > 50 => {
                println!("Clicked on the right side at ({}, {})", x, y);
            },
            WebEvent::Click { x, y } => {
                println!("Regular click at ({}, {})", x, y);
            },
            
            // Character ranges and guards
            WebEvent::KeyPress(c) if c.is_alphabetic() => {
                println!("Letter key pressed: {}", c);
            },
            WebEvent::KeyPress(c) if c.is_numeric() => {
                println!("Number key pressed: {}", c);
            },
            WebEvent::KeyPress(c) => {
                println!("Special key pressed: {}", c);
            },
            
            // String pattern matching with guards
            WebEvent::Paste(text) if text.len() > 10 => {
                println!("Long paste: {}...", &text[..10]);
            },
            WebEvent::Paste(text) => {
                println!("Short paste: {}", text);
            },
            
            _ => println!("Other event: {:?}", event),
        }
    }
    
    // Destructuring complex structures
    let points = vec![
        Point { x: 0, y: 0 },
        Point { x: 1, y: 5 },
        Point { x: 10, y: -3 },
    ];
    
    for point in points {
        match point {
            Point { x: 0, y: 0 } => println!("Origin point"),
            Point { x: 0, y } => println!("On Y-axis at y = {}", y),
            Point { x, y: 0 } => println!("On X-axis at x = {}", x),
            Point { x, y } if x == y => println!("Diagonal point ({}, {})", x, y),
            Point { x, y } => println!("Point at ({}, {})", x, y),
        }
    }
}

advanced_pattern_matching();

In [ ]:
#[derive(Debug, Clone)]
enum Shape {
    Circle { radius: f64 },
    Rectangle { width: f64, height: f64 },
    Triangle { base: f64, height: f64 },
}

impl Shape {
    // Associated function (constructor)
    fn circle(radius: f64) -> Self {
        Shape::Circle { radius }
    }
    
    fn rectangle(width: f64, height: f64) -> Self {
        Shape::Rectangle { width, height }
    }
    
    fn square(side: f64) -> Self {
        Shape::Rectangle { width: side, height: side }
    }
    
    // Methods that work on any variant
    fn area(&self) -> f64 {
        match self {
            Shape::Circle { radius } => std::f64::consts::PI * radius * radius,
            Shape::Rectangle { width, height } => width * height,
            Shape::Triangle { base, height } => 0.5 * base * height,
        }
    }
    
    fn perimeter(&self) -> f64 {
        match self {
            Shape::Circle { radius } => 2.0 * std::f64::consts::PI * radius,
            Shape::Rectangle { width, height } => 2.0 * (width + height),
            Shape::Triangle { base, height } => {
                // Assuming right triangle for simplicity
                let hypotenuse = (base * base + height * height).sqrt();
                base + height + hypotenuse
            },
        }
    }
    
    fn shape_type(&self) -> &'static str {
        match self {
            Shape::Circle { .. } => "Circle",
            Shape::Rectangle { width, height } if width == height => "Square",
            Shape::Rectangle { .. } => "Rectangle",
            Shape::Triangle { .. } => "Triangle",
        }
    }
    
    fn scale(&mut self, factor: f64) {
        match self {
            Shape::Circle { radius } => *radius *= factor,
            Shape::Rectangle { width, height } => {
                *width *= factor;
                *height *= factor;
            },
            Shape::Triangle { base, height } => {
                *base *= factor;
                *height *= factor;
            },
        }
    }
}

fn enum_methods_demo() {
    let mut shapes = vec![
        Shape::circle(5.0),
        Shape::rectangle(4.0, 6.0),
        Shape::square(3.0),
        Shape::Triangle { base: 8.0, height: 6.0 },
    ];
    
    println!("Original shapes:");
    for (i, shape) in shapes.iter().enumerate() {
        println!("{}. {} - Area: {:.2}, Perimeter: {:.2}", 
                i + 1, shape.shape_type(), shape.area(), shape.perimeter());
    }
    
    // Scale all shapes
    for shape in &mut shapes {
        shape.scale(1.5);
    }
    
    println!("\nAfter scaling by 1.5:");
    for (i, shape) in shapes.iter().enumerate() {
        println!("{}. {} - Area: {:.2}, Perimeter: {:.2}", 
                i + 1, shape.shape_type(), shape.area(), shape.perimeter());
    }
}

enum_methods_demo();

### Option and Result Deep Dive

In [ ]:
// Working with Option<T>
fn find_word_position(text: &str, word: &str) -> Option<usize> {
    text.find(word)
}

fn get_nth_word(text: &str, n: usize) -> Option<&str> {
    text.split_whitespace().nth(n)
}

// Custom Result type
#[derive(Debug)]
enum MathError {
    DivisionByZero,
    NegativeSquareRoot,
    InvalidInput,
}

impl std::fmt::Display for MathError {
    fn fmt(&self, f: &mut std::fmt::Formatter) -> std::fmt::Result {
        match self {
            MathError::DivisionByZero => write!(f, "Cannot divide by zero"),
            MathError::NegativeSquareRoot => write!(f, "Cannot take square root of negative number"),
            MathError::InvalidInput => write!(f, "Invalid input provided"),
        }
    }
}

fn safe_divide(a: f64, b: f64) -> Result<f64, MathError> {
    if b == 0.0 {
        Err(MathError::DivisionByZero)
    } else {
        Ok(a / b)
    }
}

fn safe_sqrt(x: f64) -> Result<f64, MathError> {
    if x < 0.0 {
        Err(MathError::NegativeSquareRoot)
    } else {
        Ok(x.sqrt())
    }
}

fn quadratic_formula(a: f64, b: f64, c: f64) -> Result<(f64, f64), MathError> {
    if a == 0.0 {
        return Err(MathError::InvalidInput);
    }
    
    let discriminant = b * b - 4.0 * a * c;
    let sqrt_discriminant = safe_sqrt(discriminant)?;  // ? operator
    
    let root1 = (-b + sqrt_discriminant) / (2.0 * a);
    let root2 = (-b - sqrt_discriminant) / (2.0 * a);
    
    Ok((root1, root2))
}

fn option_result_demo() {
    let text = "The quick brown fox jumps over the lazy dog";
    
    // Option examples
    println!("=== Option Examples ===");
    
    // Using match
    match find_word_position(text, "fox") {
        Some(pos) => println!("Found 'fox' at position {}", pos),
        None => println!("'fox' not found"),
    }
    
    // Using if let
    if let Some(word) = get_nth_word(text, 2) {
        println!("Third word is: {}", word);
    }
    
    // Chaining operations
    let result = get_nth_word(text, 10)
        .map(|word| word.to_uppercase())
        .unwrap_or_else(|| "NOT_FOUND".to_string());
    println!("11th word (uppercase): {}", result);
    
    println!("\n=== Result Examples ===");
    
    // Result examples
    let calculations = vec![
        (10.0, 2.0),
        (5.0, 0.0),
        (-4.0, 2.0),
    ];
    
    for (a, b) in calculations {
        match safe_divide(a, b) {
            Ok(result) => println!("{} / {} = {:.2}", a, b, result),
            Err(e) => println!("{} / {} failed: {}", a, b, e),
        }
    }
    
    // Quadratic formula examples
    println!("\n=== Quadratic Formula ===");
    let equations = vec![
        (1.0, -5.0, 6.0),   // x² - 5x + 6 = 0
        (1.0, 0.0, -4.0),   // x² - 4 = 0
        (1.0, 2.0, 5.0),    // x² + 2x + 5 = 0 (negative discriminant)
        (0.0, 1.0, 1.0),    // Not quadratic
    ];
    
    for (a, b, c) in equations {
        match quadratic_formula(a, b, c) {
            Ok((root1, root2)) => {
                println!("{}x² + {}x + {} = 0 has roots: {:.2}, {:.2}", 
                        a, b, c, root1, root2);
            },
            Err(e) => {
                println!("{}x² + {}x + {} = 0 failed: {}", a, b, c, e);
            },
        }
    }
}

option_result_demo();

In [ ]:
// TODO: Complete the traffic light state machine

#[derive(Debug, Clone, PartialEq)]
enum TrafficLight {
    Red { duration: u32 },
    Yellow { duration: u32 },
    Green { duration: u32 },
}

impl TrafficLight {
    // TODO: Create a new red light with default duration
    fn new() -> Self {
        TrafficLight::Red { duration: 30 }
    }
    
    // TODO: Get the next state in the cycle
    fn next_state(&self) -> Self {
        match self {
            TrafficLight::Red { .. } => TrafficLight::Green { duration: 45 },
            TrafficLight::Green { .. } => TrafficLight::Yellow { duration: 5 },
            TrafficLight::Yellow { .. } => TrafficLight::Red { duration: 30 },
        }
    }
    
    // TODO: Check if cars can proceed
    fn can_proceed(&self) -> bool {
        match self {
            TrafficLight::Green { .. } => true,
            _ => false,
        }
    }
    
    // TODO: Get remaining time
    fn remaining_time(&self) -> u32 {
        match self {
            TrafficLight::Red { duration } => *duration,
            TrafficLight::Yellow { duration } => *duration,
            TrafficLight::Green { duration } => *duration,
        }
    }
    
    // TODO: Tick down the timer
    fn tick(&mut self) -> bool {
        match self {
            TrafficLight::Red { duration } => {
                if *duration > 0 {
                    *duration -= 1;
                    *duration == 0
                } else {
                    true
                }
            },
            TrafficLight::Yellow { duration } => {
                if *duration > 0 {
                    *duration -= 1;
                    *duration == 0
                } else {
                    true
                }
            },
            TrafficLight::Green { duration } => {
                if *duration > 0 {
                    *duration -= 1;
                    *duration == 0
                } else {
                    true
                }
            },
        }
    }
    
    // TODO: Get color as string
    fn color(&self) -> &'static str {
        match self {
            TrafficLight::Red { .. } => "🔴 Red",
            TrafficLight::Yellow { .. } => "🟡 Yellow",
            TrafficLight::Green { .. } => "🟢 Green",
        }
    }
}

fn traffic_light_demo() {
    let mut light = TrafficLight::new();
    let mut total_time = 0;
    
    println!("Traffic Light Simulation:");
    println!("========================");
    
    // Simulate for a few cycles
    for cycle in 1..=3 {
        println!("\nCycle {}:", cycle);
        
        for _ in 0..3 {  // Go through 3 states per cycle
            println!("  {} - {} seconds remaining", 
                    light.color(), light.remaining_time());
            
            if light.can_proceed() {
                println!("    ✅ Cars can proceed");
            } else {
                println!("    ⛔ Cars must stop");
            }
            
            // Simulate time passing
            let duration = light.remaining_time();
            total_time += duration;
            
            // Move to next state
            light = light.next_state();
        }
    }
    
    println!("\nSimulation complete. Total time: {} seconds", total_time);
}

traffic_light_demo();

In [ ]:
// TODO: Implement a JSON-like data structure using enums

use std::collections::HashMap;

#[derive(Debug, Clone)]
enum JsonValue {
    Null,
    Bool(bool),
    Number(f64),
    String(String),
    Array(Vec<JsonValue>),
    Object(HashMap<String, JsonValue>),
}

impl JsonValue {
    // TODO: Constructor methods
    fn null() -> Self {
        JsonValue::Null
    }
    
    fn bool(value: bool) -> Self {
        JsonValue::Bool(value)
    }
    
    fn number(value: f64) -> Self {
        JsonValue::Number(value)
    }
    
    fn string(value: String) -> Self {
        JsonValue::String(value)
    }
    
    fn array(values: Vec<JsonValue>) -> Self {
        JsonValue::Array(values)
    }
    
    fn object(map: HashMap<String, JsonValue>) -> Self {
        JsonValue::Object(map)
    }
    
    // TODO: Type checking methods
    fn is_null(&self) -> bool {
        matches!(self, JsonValue::Null)
    }
    
    fn is_bool(&self) -> bool {
        matches!(self, JsonValue::Bool(_))
    }
    
    fn is_number(&self) -> bool {
        matches!(self, JsonValue::Number(_))
    }
    
    fn is_string(&self) -> bool {
        matches!(self, JsonValue::String(_))
    }
    
    fn is_array(&self) -> bool {
        matches!(self, JsonValue::Array(_))
    }
    
    fn is_object(&self) -> bool {
        matches!(self, JsonValue::Object(_))
    }
    
    // TODO: Value extraction methods
    fn as_bool(&self) -> Option<bool> {
        match self {
            JsonValue::Bool(b) => Some(*b),
            _ => None,
        }
    }
    
    fn as_number(&self) -> Option<f64> {
        match self {
            JsonValue::Number(n) => Some(*n),
            _ => None,
        }
    }
    
    fn as_string(&self) -> Option<&String> {
        match self {
            JsonValue::String(s) => Some(s),
            _ => None,
        }
    }
    
    // TODO: Array and object access
    fn get_index(&self, index: usize) -> Option<&JsonValue> {
        match self {
            JsonValue::Array(arr) => arr.get(index),
            _ => None,
        }
    }
    
    fn get_key(&self, key: &str) -> Option<&JsonValue> {
        match self {
            JsonValue::Object(obj) => obj.get(key),
            _ => None,
        }
    }
    
    // TODO: Pretty printing
    fn pretty_print(&self, indent: usize) -> String {
        let spaces = " ".repeat(indent);
        match self {
            JsonValue::Null => "null".to_string(),
            JsonValue::Bool(b) => b.to_string(),
            JsonValue::Number(n) => n.to_string(),
            JsonValue::String(s) => format!("\"{}\"", s),
            JsonValue::Array(arr) => {
                if arr.is_empty() {
                    "[]".to_string()
                } else {
                    let items: Vec<String> = arr.iter()
                        .map(|v| format!("{}  {}", spaces, v.pretty_print(indent + 2)))
                        .collect();
                    format!("[\n{}\n{}]", items.join(",\n"), spaces)
                }
            },
            JsonValue::Object(obj) => {
                if obj.is_empty() {
                    "{}".to_string()
                } else {
                    let items: Vec<String> = obj.iter()
                        .map(|(k, v)| format!("{}  \"{}\": {}", 
                                            spaces, k, v.pretty_print(indent + 2)))
                        .collect();
                    format!("{\n{}\n{}}", items.join(",\n"), spaces)
                }
            },
        }
    }
}

fn json_value_demo() {
    // Create a complex JSON-like structure
    let mut person = HashMap::new();
    person.insert("name".to_string(), JsonValue::string("Alice".to_string()));
    person.insert("age".to_string(), JsonValue::number(30.0));
    person.insert("is_student".to_string(), JsonValue::bool(false));
    
    let hobbies = vec![
        JsonValue::string("reading".to_string()),
        JsonValue::string("programming".to_string()),
        JsonValue::string("hiking".to_string()),
    ];
    person.insert("hobbies".to_string(), JsonValue::array(hobbies));
    
    let mut address = HashMap::new();
    address.insert("street".to_string(), JsonValue::string("123 Main St".to_string()));
    address.insert("city".to_string(), JsonValue::string("Anytown".to_string()));
    address.insert("zip".to_string(), JsonValue::number(12345.0));
    person.insert("address".to_string(), JsonValue::object(address));
    
    person.insert("spouse".to_string(), JsonValue::null());
    
    let json_person = JsonValue::object(person);
    
    println!("JSON-like structure:");
    println!("{}", json_person.pretty_print(0));
    
    // Access nested data
    println!("\nAccessing data:");
    
    if let Some(name) = json_person.get_key("name").and_then(|v| v.as_string()) {
        println!("Name: {}", name);
    }
    
    if let Some(age) = json_person.get_key("age").and_then(|v| v.as_number()) {
        println!("Age: {}", age);
    }
    
    if let Some(first_hobby) = json_person.get_key("hobbies")
        .and_then(|v| v.get_index(0))
        .and_then(|v| v.as_string()) {
        println!("First hobby: {}", first_hobby);
    }
    
    if let Some(city) = json_person.get_key("address")
        .and_then(|v| v.get_key("city"))
        .and_then(|v| v.as_string()) {
        println!("City: {}", city);
    }
    
    // Type checking
    println!("\nType checking:");
    if let Some(spouse) = json_person.get_key("spouse") {
        println!("Spouse is null: {}", spouse.is_null());
    }
}

json_value_demo();